testing jupyter notebook from vscode

In [362]:
# imports
import sys,os
import pandas as pd
import requests
import altair as alt
from altair.expr import datum
from jupyter_datatables import init_datatables_mode


In [363]:
print(f'Altair Version: {alt.__version__}')
print(f'Virtualenv: {os.getenv("VIRTUAL_ENV")}')
print(f'Python Executable: {sys.executable}')
print(f'Python Version: {sys.version}')

Altair Version: 4.1.0
Virtualenv: C:\Users\jon.galloy\PycharmProjects\VENV\DATA_VIZ
Python Executable: c:\users\jon.galloy\pycharmprojects\venv\data_viz\scripts\python.exe
Python Version: 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]


In [46]:
alt.renderers.enable('default')
init_datatables_mode()

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

In [47]:
# set up alphavantage api stuff
# symbol = input()
api_key = 'LAJBI8EN52GC18OY'
base_url = 'https://www.alphavantage.co/query?'
params = {'function': 'TIME_SERIES_DAILY_ADJUSTED',
          'symbol': 'GME',
          'apikey': api_key}

In [48]:
# get response from api and format it a bit and put it into a dict
# this helps with the mulit level nested json by 'stripping' metadata header info
response = requests.get(base_url, params=params)
response_dict = response.json()
_, header = response.json()

In [49]:
# create a pandas dataframe from response_dict
df = pd.DataFrame.from_dict(response_dict[header], orient='index')
# strip the cols names down to bare bones header
df_cols = [i.split(' ')[1] for i in df.columns]
df.columns = df_cols

In [50]:
df['date'] = df.index

In [51]:
df

<JupyterRequire.display.SafeScript object>

,open,high,low,close,adjusted,volume,dividend,split,date
2021-03-15,277.52,283.0,206.0,220.14,220.14,24226187,0.0000,1.0,2021-03-15
2021-03-12,274.9965,295.5,262.27,264.5,264.5,25845899,0.0000,1.0,2021-03-12
2021-03-11,241.64,281.5,232.6,260.0,260.0,28312493,0.0000,1.0,2021-03-11
2021-03-10,269.43,348.5,172.0,265.0,265.0,71570566,0.0000,1.0,2021-03-10
2021-03-09,217.71,249.85,208.5101,246.9,246.9,39099331,0.0000,1.0,2021-03-09
...,...,...,...,...,...,...,...,...,...
2020-10-26,14.93,15.45,13.26,13.45,13.45,13376269,0.0000,1.0,2020-10-26
2020-10-23,15.05,15.38,14.55,15.0,15.0,6507324,0.0000,1.0,2020-10-23
2020-10-22,14.2,15.87,14.19,14.91,14.91,16212232,0.0000,1.0,2020-10-22
2020-10-21,13.9,14.42,13.8,14.1,14.1,5361928,0.0000,1.0,2020-10-21


In [52]:
#this saves a new df with a new index 01234 etc not the date string.
df1 = df.reset_index(drop=True)

In [53]:
#this works now with all cols as 'objects'. no need to convert 'date' col to datetime in pandas
chart = alt.Chart(df1).mark_circle(size=50).encode(
    x='date:T',
    y='open:Q',    
    tooltip=['open','close','high','low']
).properties(width=900).interactive()


In [54]:
chart

alt.Chart(...)

In [348]:
chart = alt.Chart(df1).mark_line(point=True,color='orange',strokeDash=[5,2],).encode(
    x='date:T',
    y='open:Q',    
    tooltip=['open']
).properties(width=900).interactive()
chart

alt.Chart(...)

In [341]:
open = alt.Chart(df1).mark_line().encode(
    x='date:T',
    y='open:Q',    
    tooltip=['open']
).interactive()

close = alt.Chart(df1).mark_point().encode(
    x='date:T',
    y='close:Q',    
    tooltip=['close']    
).properties(width=900).interactive()


In [342]:
open.mark_line(point=True,strokeDash=[6,2],color='orange') + close.mark_line(point=True,strokeDash=[4,1],color='blue') 

alt.LayerChart(...)

In [289]:
line = alt.Chart(df1).mark_line(        
    color='orange',
    strokeWidth=2,
    strokeDash=[10,5],
    strokeCap='square'
).transform_window(
    rolling_mean='mean(open)',
    frame=[-15, 15]
).encode(    
    x='date:T',
    y='rolling_mean:Q'
)

points = alt.Chart(df1).mark_point(shape='diamond',size= 60, fontStyle='italic', color='purple').encode(    
    tooltip=['open','close'],
    x='date:T',
    y=alt.Y('open:Q',
            axis=alt.Axis(title='Open Price'))
).properties(width=900).interactive()



In [290]:
points + line

alt.LayerChart(...)

In [291]:
# notice the new index field and 'date' field
df1

<JupyterRequire.display.SafeScript object>

,open,high,low,close,adjusted,volume,dividend,split,date
0,277.52,283.0,206.0,220.14,220.14,24226187,0.0000,1.0,2021-03-15
1,274.9965,295.5,262.27,264.5,264.5,25845899,0.0000,1.0,2021-03-12
2,241.64,281.5,232.6,260.0,260.0,28312493,0.0000,1.0,2021-03-11
3,269.43,348.5,172.0,265.0,265.0,71570566,0.0000,1.0,2021-03-10
4,217.71,249.85,208.5101,246.9,246.9,39099331,0.0000,1.0,2021-03-09
...,...,...,...,...,...,...,...,...,...
95,14.93,15.45,13.26,13.45,13.45,13376269,0.0000,1.0,2020-10-26
96,15.05,15.38,14.55,15.0,15.0,6507324,0.0000,1.0,2020-10-23
97,14.2,15.87,14.19,14.91,14.91,16212232,0.0000,1.0,2020-10-22
98,13.9,14.42,13.8,14.1,14.1,5361928,0.0000,1.0,2020-10-21


In [292]:
# grab the 2 cols you want to get the difference for...
df_difs = df1[['open','close']]

In [293]:
# grab the other 2 cols you want to get the difference from...
df_hilo_Diff = df1[['high', 'low']]

In [294]:
df_difs

<JupyterRequire.display.SafeScript object>

,open,close
0,277.52,220.14
1,274.9965,264.5
2,241.64,260.0
3,269.43,265.0
4,217.71,246.9
...,...,...
95,14.93,13.45
96,15.05,15.0
97,14.2,14.91
98,13.9,14.1


In [295]:
df_hilo_Diff

<JupyterRequire.display.SafeScript object>

,high,low
0,283.0,206.0
1,295.5,262.27
2,281.5,232.6
3,348.5,172.0
4,249.85,208.5101
...,...,...
95,15.45,13.26
96,15.38,14.55
97,15.87,14.19
98,14.42,13.8


In [296]:
# this is computing the difference between 'open' and 'close' and 'high' and 'low' putting result in the 'close' col or the 'low' after converting from object to numeric(float64). # otherwise math won't work
df_difs = df_difs[['open', 'close']].apply(pd.to_numeric).diff(axis=1)
df_hilo_Diff = df_hilo_Diff[['high', 'low']].apply(pd.to_numeric).diff(axis=1)

In [297]:
df_diff = df_difs['close']

In [298]:
df_hilo_Diffs = df_hilo_Diff['low']

In [299]:
df_date = df1['date']

In [300]:
df_date

0     2021-03-15
1     2021-03-12
2     2021-03-11
3     2021-03-10
4     2021-03-09
         ...    
95    2020-10-26
96    2020-10-23
97    2020-10-22
98    2020-10-21
99    2020-10-20
Name: date, Length: 100, dtype: object

In [301]:
df_diff

0    -57.3800
1    -10.4965
2     18.3600
3     -4.4300
4     29.1900
       ...   
95    -1.4800
96    -0.0500
97     0.7100
98     0.2000
99    -0.1700
Name: close, Length: 100, dtype: float64

In [302]:
df_hilo_Diffs

0     -77.0000
1     -33.2300
2     -48.9000
3    -176.5000
4     -41.3399
        ...   
95     -2.1900
96     -0.8300
97     -1.6800
98     -0.6200
99     -0.4780
Name: low, Length: 100, dtype: float64

In [303]:
# put the above created pandas series(df_date, df_diff) into a pandas datatframe called posneg
df_posneg = pd.DataFrame(dict(date = df_date, diff = df_diff))

In [304]:
df_hilo = pd.DataFrame(dict(date = df_date, diff = df_hilo_Diffs))

In [305]:
df_posneg

<JupyterRequire.display.SafeScript object>

,date,diff
0,2021-03-15,-57.3800
1,2021-03-12,-10.4965
2,2021-03-11,18.3600
3,2021-03-10,-4.4300
4,2021-03-09,29.1900
...,...,...
95,2020-10-26,-1.4800
96,2020-10-23,-0.0500
97,2020-10-22,0.7100
98,2020-10-21,0.2000


In [306]:
df_hilo

<JupyterRequire.display.SafeScript object>

,date,diff
0,2021-03-15,-77.0000
1,2021-03-12,-33.2300
2,2021-03-11,-48.9000
3,2021-03-10,-176.5000
4,2021-03-09,-41.3399
...,...,...
95,2020-10-26,-2.1900
96,2020-10-23,-0.8300
97,2020-10-22,-1.6800
98,2020-10-21,-0.6200


In [358]:


chart2 = alt.Chart(df_posneg).mark_bar().encode(
    x="date:T",
    y=alt.Y('diff:Q', axis=alt.Axis(format='$', title='Change in $')
    y="diff:Q",
    tooltip=['diff','date:T'],
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("black"),  # The positive color
        alt.value("red")  # The negative color
    )).properties(width=900,height=600).interactive()

chart2


SyntaxError: invalid syntax (<ipython-input-358-95db4102c14b>, line 4)

In [353]:
chart3 = alt.Chart(df_hilo).mark_bar().encode(
    x="date:T",
    y="diff:Q",
    tooltip=['diff','date:T'],
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("darkgreen")  # The negative color
    )
).properties(width=900, title='Net diff of Daily Hi/Lo').interactive()


In [357]:
chart3.configure_title(
    fontSize=20,
    font='Georgia',
    fontStyle='italic',
    anchor='start',
    color='gray'
)

alt.Chart(...)